### Import and prepare vaccination data to merge it into Covid cases dataset

In [2]:
#### Import libraries #### 

## for data
import pandas as pd
import numpy as np

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import missingno as msno

## for statistical tests
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [4]:
# read three csv files containing all data available regarding to vaccination in Federal District
df_part0 = pd.read_csv("part-3.csv", sep = ";")
df_part1 = pd.read_csv("part-1.csv", sep = ";")
df_part2 = pd.read_csv("part-2.csv", sep = ";")

C:\Users\Marcelo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# set dtype of date variable
df_part0['vacina_dataAplicacao'] = pd.to_datetime(df_part0['vacina_dataAplicacao'])
df_part1['vacina_dataAplicacao'] = pd.to_datetime(df_part1['vacina_dataAplicacao'])
df_part2['vacina_dataAplicacao'] = pd.to_datetime(df_part2['vacina_dataAplicacao'])

In [6]:
# set date feature as index
df_part0 = df_part0.set_index(['vacina_dataAplicacao'])
df_part1 = df_part1.set_index(['vacina_dataAplicacao'])
df_part2 = df_part2.set_index(['vacina_dataAplicacao'])

In [21]:
# set correct dtypes
df = df_part0[["document_id", "vacina_descricao_dose"]]
df = df.append([df_part1[["document_id", "vacina_descricao_dose"]], df_part2[["document_id", "vacina_descricao_dose"]]])


In [22]:
# visualize data
display(df)

,document_id,vacina_descricao_dose
vacina_dataAplicacao,,
2021-07-29,bdfcf0f3-3c76-4d43-b89d-4794d4ff27de-i0b0,1ª Dose
2021-06-17,4b3982c0-2670-4eb8-b9f8-5f441ca9909c-i0b0,1ª Dose
2021-08-04,be027a38-ae18-4363-b1bc-b47737cdad2a-i0b0,2ª Dose
2021-08-16,4b462331-25fd-4306-bb90-e77b670503aa-i0b0,1ª Dose
2021-08-10,be036555-acff-4d45-a858-a7dab610755d-i0b0,1ª Dose
...,...,...
2021-04-30,a2ebb928-7cc2-43be-9098-e41205f57031-i0b0,1ª Dose
2021-04-23,a2f18bf6-bf22-4770-aec2-a1c97f051f3c-i0b0,1ª Dose
2021-10-26,a30b7733-a437-4046-8f54-fb4de931d6e2-i0b0,2ª Dose


In [24]:
# Create dummies
# vacina_descricao_dose
dummy = pd.get_dummies(df["vacina_descricao_dose"], drop_first = True)
df = pd.concat([df, dummy], axis = 1)
df.drop("vacina_descricao_dose", inplace = True, axis = 1)

In [29]:
# rename index as 'date'
df.index.names = ['date']

In [32]:
# group doses information by date
df = df.groupby(level = "date").sum()

In [35]:
# sum doses application by date
df["DoseTotal"] = df.sum(axis = 1)

In [37]:
# save dataframe
df.to_csv("vacinas.csv")